In [16]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [17]:
import pandas as pd

In [18]:
train = pd.read_csv('data/train.csv',parse_dates='datetime',index_col='datetime')
test = pd.read_csv('data/test.csv',parse_dates='datetime',index_col='datetime')

In [19]:
train['hour'] = train.index.hour
train['month'] = train.index.month
train['year'] = train.index.year - 2011
train['day_of_week'] = train.index.weekday
train[:3]

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,month,year,day_of_week
datetime,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0,3,13,16,0,1,0,5
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0,8,32,40,1,1,0,5
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0,5,27,32,2,1,0,5


In [20]:
def error(predictions,actual):
    print predictions[:10]
    print np.array(actual)[:10]
    err = 0
    for i, pred in enumerate(predictions):
        err += (np.log1p(pred)-np.log1p(actual[i]))**2
    J = (err/len(predictions))**.5
    return J

In [21]:
from sklearn import cross_validation

In [22]:
from sklearn.linear_model import ElasticNet

In [23]:
enet = ElasticNet()

In [24]:
features = ['season','holiday','workingday','weather','humidity','temp','windspeed','hour','month','year', 'day_of_week']

In [25]:
enet.fit(train[features],train['count'])


ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute='auto',
      tol=0.0001, warm_start=False)

In [26]:
predictions = enet.predict(train[features])

In [27]:
 print error(predictions, train['count'])

[-64.3042966  -60.15640446 -52.59752165 -28.3601406  -20.80125779
 -12.66974203 -22.36199042 -33.69285712   9.43427344  37.2708727 ]
[16 40 32 13  1  1  2  3  8 14]
nan


In [28]:
import numpy as np

In [29]:
kf = cross_validation.KFold(len(train),5,shuffle=True)

In [30]:
for tr, te in kf:
    enet = ElasticNet(l1_ratio=0.01)
    enet.fit(train[features].iloc[tr], train['count'].iloc[tr])
    predictions = enet.predict(train[features].iloc[te])
    predictions[predictions < 0] = 0
    print error(predictions, train['count'].iloc[te])


[   0.            0.           71.30828352    0.            0.
   83.35062503  112.7123503   153.44231867  131.58545928  175.64533944]
[40  3 36 17  9 70 76 65 61 72]
1.22637795007
[  0.00000000e+00   6.20835595e+01   1.24254310e+02   1.31437849e+02
   1.22134090e+02   1.12382424e+02   1.61802910e+02   6.99919787e-02
   7.43755095e+00   9.30296506e+01]
[ 16  56  94 106 110  35  39   6   3  59]
1.27061384606
[  15.63034078  129.60672868   54.12753535   56.33882096  204.400881
  210.65991937   34.09663602   37.10386101   46.58297192   77.73700959]
[  8  36   2   1  30  22  30  64 154  88]
1.27431750855
[   0.            0.          122.58543936  134.09972461   72.51581649
   63.73196491  104.39321656  195.27549543  195.01892889  189.85528845]
[13  1 37 34  8 20 93 53 31  8]
1.27483473106
[   0.            0.            0.           43.75914239   95.01131044
  113.34892884  126.38773923  125.96482319    0.           52.90638924]
[32  1  2 14 84 93 67 28 17 53]
1.23974983927


In [31]:
type(predictions)

numpy.ndarray

In [32]:
predictions[np.abs(predictions) < 0] = 0

In [33]:
from sklearn.linear_model import ElasticNetCV

In [34]:
enet = ElasticNetCV()
enet.fit(train[features], train['count'])
predictions = enet.predict(train[features])
predictions[predictions < 0] = 0
print error(predictions, train['count'])

[  0.           0.           0.           0.           0.           0.           0.
   0.          17.60292517  45.34450104]
[16 40 32 13  1  1  2  3  8 14]
1.25606388483


In [35]:
for tr, te in kf:
    enet = ElasticNetCV()
    enet.fit(train[features].iloc[tr], train['count'].iloc[tr])
    predictions = enet.predict(train[features].iloc[te])
    predictions[predictions < 0] = 0
    print error(predictions, train['count'].iloc[te])

[   0.            0.           72.39085798    0.            0.
   85.14414747  115.34164943  154.48735619  133.74813715  177.86245276]
[40  3 36 17  9 70 76 65 61 72]
1.22537134609
[   0.           62.69509963  125.72454366  132.89825818  123.43162023
  114.30545311  162.56171356    1.6346668     8.9568041    95.20969236]
[ 16  56  94 106 110  35  39   6   3  59]
1.26985770438
[  17.16646852  130.48578566   56.28428323   57.98797251  204.67968714
  211.16008967   36.48737093   39.92344891   49.13498389   80.09981778]
[  8  36   2   1  30  22  30  64 154  88]
1.27395181406
[   0.            0.          124.27548754  135.25407981   74.75750628
   65.25955842  105.73782317  196.54054105  195.5699553   190.66195678]
[13  1 37 34  8 20 93 53 31  8]
1.27609234327
[   0.            0.            0.           45.09362277   95.50932293
  114.39131159  127.42469374  126.75389925    0.           54.15171242]
[32  1  2 14 84 93 67 28 17 53]
1.23791634454


In [36]:
features = ['season','workingday','weather','humidity','windspeed','hour','month','year', 'day_of_week']

In [37]:
enet

ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto', tol=0.0001,
       verbose=0)

In [38]:
p = enet.predict(test[features])

KeyError: "['hour' 'month' 'year' 'day_of_week'] not in index"

In [ ]:
test['hour'] = test.index.hour
test['month'] = test.index.month
test['year'] = test.index.year - 2011
test['day_of_week'] = test.index.weekday

In [ ]:
tdf = pd.DataFrame([test.index, p]).transpose()

In [ ]:
p[p < 0] = 0

In [ ]:
tdf.to_csv('results_enet.csv', index=False)

In [ ]:
tdf.columns = ['datetime', 'count']

In [ ]:
tdf

In [ ]:
tdf.columns

In [ ]:
tdf

In [ ]:
%pylab inline

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
clf = GradientBoostingRegressor()

In [ ]:
for tr, te in kf:
    clf = GradientBoostingRegressor()
    clf.fit(train[features].iloc[tr], train['count'].iloc[tr])
    predictions = clf.predict(train[features].iloc[te])
    predictions[predictions < 0] = 0
    print error(predictions, train['count'].iloc[te])

In [ ]:
test['hour'] = test.index.hour
test['month'] = test.index.month
test['year'] = test.index.year - 2011
test['day_of_week'] = test.index.weekday

In [ ]:
tdf = pd.DataFrame([test.index, p]).transpose()

In [ ]:
p[p < 0] = 0

In [ ]:
tdf.columns = ['datetime', 'count']

In [ ]:
tdf.to_csv('results_gbr.csv', index=False)

In [ ]:
p = clf.predict(test[features])